In [1]:
%pip install simcse

Note: you may need to restart the kernel to use updated packages.


You should consider upgrading via the 'C:\Users\Kevin\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip' command.


In [2]:
import pandas as pd
from tqdm import tqdm

from simcse import SimCSE

Lettura del dataset e creazione del modello SimCSE

In [3]:
df = pd.read_csv('./datasets/combined.csv', sep=",")

print(f"NROWS: {len(df)}")

model = SimCSE("princeton-nlp/sup-simcse-bert-base-uncased")


NROWS: 7330


Costruiamo l'indice con i termini meddra

In [4]:
model.build_index(df['meddra'].unique().tolist())

06/01/2022 20:20:37 - INFO - faiss.loader -   Loading faiss with AVX2 support.
06/01/2022 20:20:37 - INFO - faiss.loader -   Could not load library with AVX2 support due to:
ModuleNotFoundError("No module named 'faiss.swigfaiss_avx2'")
06/01/2022 20:20:37 - INFO - faiss.loader -   Loading faiss.
06/01/2022 20:20:38 - INFO - faiss.loader -   Successfully loaded faiss.
06/01/2022 20:20:38 - INFO - simcse.tool -   Encoding embeddings for sentences...
100%|██████████| 16/16 [00:27<00:00,  1.70s/it]
06/01/2022 20:21:05 - INFO - simcse.tool -   Building index...
06/01/2022 20:21:05 - INFO - simcse.tool -   Use CPU-version faiss
06/01/2022 20:21:05 - INFO - simcse.tool -   Finished


In [5]:
results = model.search(df['ade'].tolist(), top_k=5)

100%|██████████| 115/115 [04:29<00:00,  2.35s/it]


Vediamo quanti ade vengono associati al giusto meddra e quante volte il meddra corretto appare nei 5 meddra più simili all'ade

In [9]:
ades = df['ade'].unique().tolist()
real_meddras = df['meddra'].tolist()
predicted_meddras = []
for result_list in results:
    res = []
    for (meddra, _) in result_list:
        res.append(meddra)
    predicted_meddras.append(res)

to_classify = df.shape[0]
correctly_classified = 0
correct_meddra_in_top_5 = 0
wrongly_classified = 0

for (real_meddra, predicted_meddra_options) in tqdm(zip(real_meddras, predicted_meddras)):
    if predicted_meddra_options:
        if predicted_meddra_options[0] == real_meddra:
            correctly_classified += 1
        elif real_meddra in predicted_meddra_options:
            correct_meddra_in_top_5 += 1
        else:
            wrongly_classified += 1
    else:
        wrongly_classified += 1

print()
print(f"Examples: {to_classify}")
print(f"Correctly classified: {correctly_classified}")
print(f"Correct meddra is in the top 5: {correct_meddra_in_top_5}")

7330it [00:00, 1831759.31it/s]


Examples: 7330
Correctly classified: 3227
Correct meddra is in the top 5: 1215


In [10]:
print(len(df[df['ade'] == df['meddra']]))

1444
